### [트랜스 포머](https://arxiv.org/pdf/1706.03762)
![transformers](https://i.sstatic.net/BpxYv.png)
- **Encoder** + **Decoder** : t5
- **Encoder** 모델 : Bert 류 등
    * 양방향 어텐션(bidirectional attention)
- **Decoder** 모델 : gpt 류 등
    * 자기 회귀 어텐션(autoregressive attention)


### **어텐션 가중치(attention weights)**
- 선형 결합을 통한, 전체 시퀀스의 가중 평균
    $$ x'_{i} = \sum_{j=1}^{n} w_{ji}x_{j}$$
    $ x'_{i} : \text{linear combination} $ (선형 결합)<Br>
    계수(coefficient) 는 $ w_{ji} : \text{attention weights} $ 총합이 1
    
    선형 결합으로 생성된 임베딩 : **문맥 고려 임베딩(contextualized embedding)**

### **셀프 어텐션**

![image.png](https://img.notionusercontent.com/s3/prod-files-secure%2F1f406a34-9833-41b3-af5a-06ebe0821494%2F21f6d610-d50d-4e43-b38d-27b010a1153a%2F1_self_attention.svg/size/?exp=1731833224&sig=-78EuHKYI03NfVZHf57uJkKR1m8ghm-5BOYRelCDV0U)

- **스케일드 닷 어텐션(점곱 어텐션)** 
    1. positional embedding 생성(위치정보)
    2. 쿼리(Q), 키(K), 밸류(V)의 $ W_Q , W_K ,  W_V $와 곱해져 프로젝션
        $$ Q = XW_Q, \quad K = XW_K, \quad V = XW_V $$
    
    3. **어텐션 점수** 계산 : **쿼리(Q)** 와 **키(K)의** **내적(dot product)**
        $$ \text{score}(i,j) = Q_i \cdot K_j^T $$
        $$\text{score\_scaled}(i, j) = \frac{Q_i \cdot K_j^T}{\sqrt{d_k}}$$
        - 안정성 추가 : $\sqrt{d_k}$ 
          - $d_k$:입력 벡터의 차원 수
    
    3. **어텐션 가중치(attention weights) 계산**:
        - 스케일된 점수를 소프트맥스 함수를 통해 변환하여 어텐션 가중치를 얻습니다:
          $$
          w_{ij} = \text{softmax}(\text{score\_scaled}(i, j)) \\
          = \text{softmax}(\frac{Q_i \cdot K_j^T}{\sqrt{d_k}})
          $$
    
    4. **토큰 임베딩 업데이트 (어텐션 출력)**:
        - 어텐션 가중치를 통해 밸류(V)의 가중 평균을 계산하여 새로운 출력 벡터 생성
          $$ x'_{i} = \sum_{j=1}^{n} w_{ij} v_{j} $$
          $$ \therefore x'_i \text{(output vector)} = \sum_{j=1}^{n} \text{softmax}\left(\frac{Q_i \cdot K_j^T}{\sqrt{d_k}}\right) V_j $$
          - $i$ = 쿼리 벡터의 인덱스 - 즉 각 쿼리마다의 어텐션 출력인 w'_i 이다.
          - $j$ = 키 벡터의 인덱스
          - $d_k$ = 키 벡터의 차원